Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [78]:
import numpy as np
import cv2

# Repeatability (cosistent results)
np.random.seed(0)

# video frame size
VFILENAME = "walking.mp4"
HEIGHT = 406
WIDTH = 722

Load video frames from file

In [79]:
def get_frames(filename):
    video = cv2.VideoCapture(filename) # return video capture object
    while video.isOpened():
        ret,frame = video.read() # read the frame
        if ret: 
            yield frame       # if the flag is valid: iteratively yielding frames 
        else:
            break            # if not good; break the loob and release resources      
    video.release()
    yield None
        

Creating a particle cloud

In [80]:
# assume number of particles 
NUM_PARTICLES = 150
# velocity range: assume a particle is not moving faster than half a pixel
VEL_RANGE = 0.5
def initialize_particles():
    # create particles array: fill out with random numbers from 0 to 1, one row per particle, 4 columns
    particles = np.random.rand(NUM_PARTICLES, 4)
    # scale the numbers 
    particles = particles * np.array( (WIDTH, HEIGHT, VEL_RANGE, VEL_RANGE))
    # centered the velocity to 0: shift last two columns a bit 
    particles[:,2:4] -= VEL_RANGE/2.0
    print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [81]:
def apply_velocity(particles):
    # first col increment with x direction velocity component of 3rd col: aka index 2
    particles[:,0] += particles[:,2]
    # similarly for y 
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [82]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        # the new value for x cordinate:set an upper and lower bound
        particles[i,0] = max(0, min(WIDTH-1, particles[i,0]))
        # similarly for y
        particles[i,1] = max(0, min(HEIGHT-1, particles[i,1]))
        # now we have cloud of particles clever enoght not to fall off the edge
    return particles

Measure each particle's quality

In [83]:
# color pixels representing the whole target: not to bright not to dark
TARGET_COLOUR = np.array((156, 74, 38))

def compute_errors(particles, frame):
    # create numpy array to store those color differences: fill it with zeros to start with
    errors = np.zeros(NUM_PARTICLES)
    # loop over all the particles
    for i in range(NUM_PARTICLES):
        # cast the x, y position to integr
        x = int(particles[i,0])
        y = int(particles[i,1])
        # pull out the pixels values at this position
        pixel_colour = frame[y, x, : ]
        # single value to represent the colour DIFFERENCE at this pixels: MEAN SQUARE DIFFERENCE
        errors[i] = np.sum ( (TARGET_COLOUR - pixel_colour)**2 )
    return errors

Assign weights to the particles based on their quality of match

In [84]:
def compute_weights(errors):
    # invert the errors: we want low errors get high weight
    weights = np.max(errors) - errors
    # prevent the particles from piling up along the edge: set their weight to zero
    weights[
        # along the left hand edge of the frame
         (particles[: , 0] == 0) | 
        # the right edge of the frame
         (particles[:, 0] == WIDTH-1) | 
        # same for y
         (particles[: , 1] == 0) |  
        # bottom edge
         (particles[:, 1] == HEIGHT-1) 
    ] = 0.0
    weights = weights**4
    return weights

Resample particles according to their weights

In [85]:
def resample(particles, weights):
    # normalize the weights to sum 1: use the probability distribution over the particles
    probabilities = weights / np.sum(weights)
    # resample particles acc to these prob: build new particle array by sampling from the current particles (ones with higher rate will got chosen many times)
    index_numbers = np.random.choice(
        NUM_PARTICLES, 
        size=NUM_PARTICLES, 
        p=probabilities)
    # rebuild the particles array acc to these indexes
    particles = particles[index_numbers, :]
    
    # single best guess x,y position: mean over all values
    x = np.mean(particles[:,0])
    y = np.mean(particles[:,1])
    
    # return particles array and a tuple of x, y
    return particles,(int(x), int(y))

Fuzz the particles

In [86]:
# specify the standard deviation
POS_SIGMA = 1.0
VEL_SIGMA = 0.5

def apply_noise(particles):
    # create noise at one column at atime
    noise = np.concatenate(
    (
       np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES, 1)),
       np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES, 1)),
       np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES, 1)),
       np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES, 1)),                                  
    ),
    axis=1)
    particles += noise
    
    return particles

Display the video frames

In [87]:
def display(frame, particles, location):
    if len(particles) > 0:  # check if there are any particles to display
        for i in range(NUM_PARTICLES):  # iterate through them
            x = int(particles[i,0])   # cast them into intgers: use the values as pixels coordinates
            y = int(particles[i,1])  # similarly for the y values  
            cv2.circle(frame, (x,y), 1, (0,255,1), 1) # draw particles as tiny circle on top of video frame
            # use circle function(frame, circle center, radius, color: bgr not rgb green, set small 1 pixel)
    if len(location) > 0: # location is our best guest where a target is
        cv2.circle(frame, location, 15, (0,0,255), 5) 
            # slightly larger radius:15, and red color
    cv2.imshow('frame', frame) # display the video frame
    if cv2.waitKey(30)== 27:   # wait 30 ms to see if the user hit escape key
        if cv2.waitKey(0) == 27: # pause there
            return True # playback should stop if the user hit the escape key twice
    return False

Main routine

In [88]:
particles = initialize_particles()

for frame in get_frames(VFILENAME):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, location)
    if terminate:
        break
cv2.destroyAllWindows()


[[ 3.96243350e+02  2.90366883e+02  5.13816880e-02  2.24415915e-02]
 [ 3.05878765e+02  2.62233010e+02 -3.12063944e-02  1.95886500e-01]
 [ 6.95764513e+02  1.55677257e+02  1.45862519e-01  1.44474599e-02]
 [ 4.10128173e+02  3.75792235e+02 -2.14481971e-01 -2.06435350e-01]
 [ 1.45976830e+01  3.38043657e+02  1.39078375e-01  1.85006074e-01]
 [ 7.06562443e+02  3.24458377e+02 -1.92603189e-02  1.40264588e-01]
 [ 8.53941355e+01  2.59807935e+02 -1.78323356e-01  2.22334459e-01]
 [ 3.76774488e+02  1.68352748e+02 -1.17722194e-01  1.37116845e-01]
 [ 3.29340540e+02  2.30784183e+02 -2.40605100e-01  5.88177485e-02]
 [ 4.41933112e+02  2.50475203e+02  2.21874039e-01  9.09101496e-02]
 [ 2.59564704e+02  1.77434973e+02  9.88155980e-02 -2.19887264e-01]
 [ 4.81405569e+02  2.72278975e+02 -1.44808719e-01 -1.85536851e-01]
 [ 2.27739269e+02  1.47666573e+02  3.50983852e-02 -3.06992433e-02]
 [ 7.13605911e+02  4.14301932e+01 -1.45561622e-01 -1.69345241e-01]
 [ 4.71544211e+02  1.02836391e+02 -1.68446136e-02 -1.27787204e